In [1]:
import graphviz

# Crear el objeto del grafo
dot = graphviz.Digraph('flujo_main', comment='Diagrama de Flujo de main.py')
dot.attr(rankdir='TB', splines='ortho', label='Diagrama de Flujo Lógico de main.py', fontsize='20')

# Definir estilos para los nodos
action_style = {'shape': 'box', 'style': 'rounded,filled', 'fillcolor': 'lightblue'}
decision_style = {'shape': 'diamond', 'style': 'filled', 'fillcolor': 'lightyellow'}
io_style = {'shape': 'parallelogram', 'style': 'filled', 'fillcolor': 'lightgrey'}
start_end_style = {'shape': 'ellipse', 'style': 'filled', 'fillcolor': 'palegreen'}

# 1. Nodos del Flujo
# --- Setup ---
dot.node('start', 'Inicio', **start_end_style)
dot.node('setup', 'Configuración Inicial\n(Cargar Modelo, Conectar DB/Arduino, Iniciar Cámara)', **action_style)

# --- Bucle Principal ---
dot.node('loop_start', 'Bucle Principal (while True)', shape='box', style='dashed')
dot.node('captura', 'Capturar Frame de Cámara', **io_style)
dot.node('yolo', 'Detectar Placa (YOLO)', **action_style)
dot.node('check_detection', '¿Placa Detectada?', **decision_style)

# --- Lógica de Estabilización ---
dot.node('start_timer', 'Iniciar Temporizador (3s)', **action_style)
dot.node('check_timer', '¿Pasaron 3 seg?', **decision_style)
dot.node('update_roi', 'Actualizar ROI Estabilizada', **action_style)

# --- Procesamiento OCR ---
dot.node('process_roi', 'Preprocesar ROI y Reconocer Texto (OCR)', **action_style)
dot.node('check_ocr', '¿Texto Reconocido?', **decision_style)

# --- Lógica de Negocio ---
dot.node('check_db', 'Verificar Placa en DB', **action_style)
dot.node('check_auth', '¿Placa Registrada?', **decision_style)
dot.node('open_gate', 'Abrir Barrera y Guardar Registro', **action_style)
dot.node('show_gui', 'Mostrar GUI de Registro', **action_style)

# --- Finalización y Reintentos ---
dot.node('reset_success', 'Reiniciar Detección (Pausa 5s)', **action_style)
dot.node('reset_fail', 'Reiniciar Detección (Pausa 2s)', **action_style)
dot.node('check_quit', '¿Usuario presiona "q"?', **decision_style)
dot.node('end', 'Finalizar\n(Liberar Cámara, Cerrar Conexiones)', **start_end_style)


# 2. Conexiones del Flujo
dot.edge('start', 'setup')
dot.edge('setup', 'loop_start')
dot.edge('loop_start', 'captura')
dot.edge('captura', 'yolo')
dot.edge('yolo', 'check_detection')

# Flujo si no se detecta placa
dot.edge('check_detection', 'check_quit', label='No')

# Flujo si se detecta placa
dot.edge('check_detection', 'start_timer', label='Sí (Primera vez)')
dot.edge('start_timer', 'check_timer')
dot.edge('check_detection', 'check_timer', label='Sí (Ya detectada)')

# Flujo del temporizador
dot.edge('check_timer', 'update_roi', label='No (< 3s)')
dot.edge('update_roi', 'check_quit')
dot.edge('check_timer', 'process_roi', label='Sí (>= 3s)')

# Flujo del OCR
dot.edge('process_roi', 'check_ocr')
dot.edge('check_ocr', 'check_db', label='Sí')
dot.edge('check_ocr', 'reset_fail', label='No')

# Flujo de la Base de Datos
dot.edge('check_db', 'check_auth')
dot.edge('check_auth', 'open_gate', label='Sí (Autorizada)')
dot.edge('check_auth', 'show_gui', label='No (No Registrada)')
dot.edge('open_gate', 'reset_success')
dot.edge('show_gui', 'reset_success')

# Flujos de reinicio y salida
dot.edge('reset_success', 'check_quit')
dot.edge('reset_fail', 'check_quit')
dot.edge('check_quit', 'loop_start', label='No')
dot.edge('check_quit', 'end', label='Sí')


# 3. Generar y guardar el diagrama
try:
    dot.render('flujo_main', format='pdf', view=True, cleanup=True)
    print("\n✅ ¡Diagrama 'flujo_main.pdf' generado con éxito!")
except graphviz.backend.execute.ExecutableNotFound:
    print("\n❌ Error: Graphviz no está instalado en tu sistema.")
    print("Recuerda ejecutar: sudo apt-get install graphviz")


✅ ¡Diagrama 'flujo_main.pdf' generado con éxito!


In [2]:
import graphviz

dot = graphviz.Digraph('arquitectura_sistema_ocr_v2', comment='Diagrama de Flujo con Sensor de Proximidad')
dot.attr(rankdir='TB', splines='ortho', label='Arquitectura con Control de Proximidad', fontsize='20')

# Definir estilos
action_style = {'shape': 'box', 'style': 'rounded,filled', 'fillcolor': 'lightblue'}
decision_style = {'shape': 'diamond', 'style': 'filled', 'fillcolor': 'lightyellow'}
io_style = {'shape': 'parallelogram', 'style': 'filled', 'fillcolor': 'lightgrey'}
start_end_style = {'shape': 'ellipse', 'style': 'filled', 'fillcolor': 'palegreen'}
control_style = {'shape': 'box', 'style': 'rounded,filled', 'fillcolor': '#a2d2ff'} # Color para el bloque de control

# Nodos
with dot.subgraph(name='cluster_hardware') as c:
    c.attr(style='filled', color='lightgrey', label='Hardware y Sensores')
    c.node('prox_sensor', '➡️ Sensor de Proximidad', shape='box') # <-- NODO NUEVO
    c.node('camara', '📷 Cámara', shape='box')
    c.node('arduino', '🤖 Controlador Arduino', shape='box')
    c.node('flash', '💡 Flash / Iluminador', shape='box')
    c.node('talanquera', '🚧 Talanquera / Barrera', shape='box')

with dot.subgraph(name='cluster_software') as c:
    c.attr(label='Procesamiento de Software (Python)')
    c.node('listener', 'Esperar Señal de "CARRO_DETECTADO"', **io_style) # <-- LÓGICA NUEVA
    c.node('ocr_pipeline', 'Pipeline de OCR\n(Captura -> YOLO -> Preproc -> Tesseract)', **action_style)
    c.node('db_logic', 'Lógica de Base de Datos\n(Verificar/Registrar Placa)', **action_style)
    c.node('db', 'Base de Datos (SQL Server)', shape='cylinder')

with dot.subgraph(name='cluster_control') as c:
    c.attr(label='Control de Luminosidad (Opcional/Avanzado)', style='filled', color='#bde0fe')
    c.node('fuzzy_control', 'Controlador de Luminosidad (Fuzzy/RL)')


# Conexiones
dot.edge('prox_sensor', 'arduino')
dot.edge('arduino', 'listener', label='Señal "CARRO_DETECTADO"') # <-- DISPARADOR PRINCIPAL

# El bucle de procesamiento solo se activa después de la señal
dot.edge('listener', 'ocr_pipeline', label='Activar')
dot.edge('ocr_pipeline', 'db_logic', label='Texto de Placa')
dot.edge('db_logic', 'db')

# Salidas
dot.edge('db_logic', 'arduino', label='Comando "OPEN"')
dot.edge('arduino', 'talanquera')

# Bucle de control de luz (ahora parte del pipeline)
dot.edge('ocr_pipeline', 'fuzzy_control', label='Confianza OCR')
dot.edge('fuzzy_control', 'arduino', label='Ajuste de Flash')
dot.edge('arduino', 'flash')

# Regreso al estado de espera
dot.edge('db_logic', 'listener', label='Resetear y Esperar\nSiguiente Vehículo', style='dashed')


# Generar y guardar el diagrama
try:
    dot.render('arquitectura_con_proximidad', format='pdf', view=True, cleanup=True)
    print("\n✅ ¡Diagrama 'arquitectura_con_proximidad.pdf' generado con éxito!")
except Exception as e:
    print(f"\n❌ Error al generar el diagrama: {e}")
    print("Asegúrate de que Graphviz esté instalado en tu sistema.")


✅ ¡Diagrama 'arquitectura_con_proximidad.pdf' generado con éxito!
